In [11]:
import numpy as np
from scipy.optimize import minimize

# Parameters for the toy system
k = 1.0  # Spring constant
m = 1.0  # Mass
h = 0.1  # Time step

# Generate toy data
def true_trajectory(q0, dq0, steps):
    trajectory = []
    q, dq = q0, dq0
    for _ in range(steps):
        ddq = -(k / m) * q  # Harmonic oscillator dynamics
        trajectory.append((q, dq))
        dq += ddq * h
        q += dq * h
    return trajectory

# Euler-Lagrange residuals for the discretized system
def discrete_lagrangian(q1, q2, h, alpha):
    q_mid = (q1 + q2) / 2
    dq = (q2 - q1) / h
    # Approximating Lagrangian as a polynomial: L = alpha[0]*q^2 + alpha[1]*dq^2
    return alpha[0] * q_mid**2 + alpha[1] * dq**2

def del_residual(q1, q2, q3, h, alpha):
    Ld1 = discrete_lagrangian(q1, q2, h, alpha)
    Ld2 = discrete_lagrangian(q2, q3, h, alpha)
    return (Ld2 - Ld1) ** 2  # Residual square

# Loss function to minimize residuals
def loss_function(alpha, trajectory, h, lam=1e-3):
    loss = 0
    for i in range(len(trajectory) - 2):
        q1, q2, q3 = trajectory[i][0], trajectory[i+1][0], trajectory[i+2][0]
        loss += del_residual(q1, q2, q3, h, alpha)
    # Regularization to avoid large alphas
    loss += lam * np.sum(alpha**2)
    return loss

# Generate a simple trajectory
trajectory = true_trajectory(q0=1.0, dq0=0.0, steps=50)

# Initial guess for Lagrangian coefficients
alpha_init = np.array([100, 100])  # Start with a guess

# Minimize residuals
result = minimize(
    loss_function,
    alpha_init,
    args=(trajectory, h),
    method='CG',
    options={'disp': True}
)

print("Optimized alpha:", result.x)


Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 16
         Function evaluations: 105
         Gradient evaluations: 35
Optimized alpha: [0.00117459 0.00117612]
